In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
from requests import get, request, post
from requests.compat import urlparse, urljoin
from urllib.request import urlopen, Request
import re, json
import requests


In [ ]:
def product_list(id_url, li, depth): # 카테고리 ID를 사용하여 제품 목록을 가져오는 함수
    url = "https://sik.search.blue.cdtapps.com/kr/ko/search"
    category_id = id_url.split('-')[-1][:-1]

    for i in range(1, 10):
        page = i
        offset = (page - 1) * 24

        #API 요청에 사용할 페이로드 정의
        payload = {
            "searchParameters": {
                "input": category_id,  # category ID
                "type": "CATEGORY"
            },
            "isUserLoggedIn": False,
            "components": [{
                "component": "PRIMARY_AREA",
                "columns": 4,
                "allVariants": False,
                "filterConfig": {"max-num-filters": 2},
                "sort": "RELEVANCE",
                "types": {
                    "main": "PRODUCT",
                    "breakouts": [
                        "PLANNER", "LOGIN_REMINDER",
                        "MATTRESS_WARRANTY", "FINANCIAL_SERVICES"
                    ]
                },
                "window": {"offset": offset, "size": 24}
            }]
        }

        resp = requests.post(url, headers=header, data=json.dumps(payload))
        data = resp.json()

        if len(data['results']) == 0: # 더 이상 제품이 없으면 중단
            break

        # 제품 URL 추출
        for j in range(len(data['results'][0]['items'])):
            for d in data['results'][0]['items'][j]:
                if 'product' not in d:
                    continue
                li.append((depth+1, data['results'][0]['items'][j]['product']['pipUrl']))


In [ ]:
URLs = list()
Seen = list()

# 크롤링 시작 지점
URLs.append((0,'https://www.ikea.com/kr/ko/cat/lighting-li001/'))

while URLs:
    row = URLs.pop(-1)
    depth = row[0]
    url = row[1]

    if row in URLs or row in Seen:
        continue
    Seen.append(row)

    print(f'this turn! => Depth:{depth}, {url}')

    resp = get(url, headers=headers)
    if resp.status_code != 200:
        continue

    if depth < 2:
        if re.search('text/html', resp.headers['content-type']):
            dom = BeautifulSoup(resp.text, 'html.parser')
            for a in dom.select('main ul[class] > li[class] > a[href]'):
                if not re.search(r"%?%", a.attrs['href']) and not re.search(r"/f/", a.attrs['href']):
                    URLs.append((depth+1, a.attrs['href']))
    elif depth == 2:
        product_list(url, URLs, depth)

    elif depth == 3:
        if re.search('text/html', resp.headers['content-type']):
            dom = BeautifulSoup(resp.text, 'html.parser')
            for img in dom.select('img[src]'): # <img> 태그 선택
                if re.search(r".*kr.*s$", img.attrs['src']):
                    URLs.append((depth+1, img.attrs['src']))

    if re.search('image/jpeg', resp.headers['content-type']): # JPEG인 경우 저장
        filename = (url.split('/')[-1]).split('?')[0]
        with open(f'/content/drive/MyDrive/ikea_img/{filename}', 'wb') as fp:
            fp.write(resp.content)

this turn! => Depth:0, https://www.ikea.com/kr/ko/cat/lighting-li001/
this turn! => Depth:1, https://www.ikea.com/kr/ko/cat/bathroom-lighting-10736/
this turn! => Depth:2, https://www.ikea.com/kr/ko/cat/mirrors-with-lights-49138/
this turn! => Depth:3, https://www.ikea.com/kr/ko/p/faxaelven-mirror-cabinet-w-built-in-lighting-white-s89525740/
this turn! => Depth:4, https://www.ikea.com/kr/ko/images/products/rodret-wireless-dimmer-power-switch-smart-white__1180657_pe896397_s5.jpg?f=xxxs
this turn! => Depth:4, https://www.ikea.com/kr/ko/images/products/faxaelven-mirror-cabinet-w-built-in-lighting-white__1356215_pe953326_s5.jpg?f=s
this turn! => Depth:4, https://www.ikea.com/kr/ko/images/products/faxaelven-mirror-cabinet-w-built-in-lighting-white__1330517_pe945710_s5.jpg?f=s
this turn! => Depth:4, https://www.ikea.com/kr/ko/images/products/faxaelven-mirror-cabinet-w-built-in-lighting-white__1223049_pe914418_s5.jpg?f=s
this turn! => Depth:4, https://www.ikea.com/kr/ko/images/products/faxael

[]